In [3]:
from scipy import signal
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.integrate import simps
import csv
from itertools import zip_longest


def CheckCorrelation(User,Band,Label):
    file_path = os.path.join("ChannelsExtracted",User, Label+".csv")
    data = np.loadtxt(fname=file_path)
    print(np.corrcoef(data, Band))
    
def ExtractFeature(low , high , UserName,Channel):
    win = 256
    AbsoluteBandArray=[]
    for i in range(40):
        WholeCsv = np.genfromtxt("ChannelsExtracted/"+str(UserName)+"/"+"3035"+str(i)+"featuresraw.csv", delimiter=",")
        data = WholeCsv[:,ParseChannel(Channel)]
        freqs, psd = signal.welch(data, 128, nperseg=win)
        # Plot the power spectrum
        #sns.set(font_scale=1.2, style='white')
        #plt.figure(figsize=(8, 4))
        #plt.plot(freqs, psd, color='k', lw=2)
        #plt.xlabel('Frequency (Hz)')
        #plt.ylabel('Power spectral density (V^2 / Hz)')
        #plt.ylim([0, psd.max() * 1.1])
        #plt.title("Welch's periodogram "+str(i))
        #plt.xlim([0, freqs.max()])
        #sns.despine()
        # Define alpha lower and upper limits
        # Find intersecting values in frequency vector
        idx_delta = np.logical_and(freqs >= low, freqs <= high)
        # Plot the power spectral density and fill the delta area
        #plt.figure(figsize=(7, 4))
        #plt.plot(freqs, psd, lw=2, color='k')
        #plt.fill_between(freqs, psd, where=idx_delta, color='skyblue')
        #plt.xlabel('Frequency (Hz)')
        #plt.ylabel('Power spectral density (uV^2 / Hz)')
        #plt.ylim([0, psd.max() * 1.1])
        #plt.title("Welch's periodogram")
        #sns.despine()
        # Frequency resolution
        freq_res = freqs[1] - freqs[0] 
        # Compute the absolute power by approximating the area under the curve
        delta_power = simps(psd[idx_delta], dx=freq_res)
        AbsoluteBandArray.append(delta_power)
        #print(AbsoluteBandArray[i])
        total_power = simps(psd, dx=freq_res)
        delta_rel_power = delta_power / total_power
    #print(len(AbsoluteBandArray))
    return AbsoluteBandArray;

def ExtractByNumberAndWindow(low , high , UserName,Channel,win):
    AbsoluteBandArray=[]
    for i in range(40):
        WholeCsv = np.genfromtxt("ChannelsExtracted/s01/"+str(i)+"featuresraw.csv", delimiter=",")
        data = WholeCsv[:,Channel]
        freqs, psd = signal.welch(data, 128, nperseg=win)
        idx_delta = np.logical_and(freqs >= low, freqs <= high)
        freq_res = freqs[1] - freqs[0] 
        # Compute the absolute power by approximating the area under the curve
        delta_power = simps(psd[idx_delta], dx=freq_res)
        AbsoluteBandArray.append(delta_power)
        total_power = simps(psd, dx=freq_res)
        delta_rel_power = delta_power / total_power
    return AbsoluteBandArray;
 
def AllUsersFeature(low,high,Channel):
    AbsoluteBandArrays=[[]]
    for i in range(32):
        if i < 10:
             name = '%0*d' % (2,i+1)
        else:
             name = i+1
        AbsoluteBandArrays += [ExtractFeature(low , high , "s"+str(name),Channel)]
        
def UserOneChannelAllBands(UserName,ChannelNumber,Window):
    AbsoluteTheta=ExtractByNumberAndWindow(3 , 7 , UserName,ChannelNumber,Window)
    AbsoluteAlpha=ExtractByNumberAndWindow(8 , 13 , UserName,ChannelNumber,Window)
    AbsoluteBeta=ExtractByNumberAndWindow(14 , 29 , UserName,ChannelNumber,Window)
    AbsoluteGamma=ExtractByNumberAndWindow(30 , 47 , UserName,ChannelNumber,Window)
    AllBands=[]
    AllBands+=[AbsoluteTheta]
    AllBands+=[AbsoluteAlpha]
    AllBands+=[AbsoluteBeta]
    AllBands+=[AbsoluteGamma]
    return AllBands;

def OneUserAllChannelsAllBandsAllVid(UserName,Window):
    AllBands=[]
    for i in range(32):
        print(i)
        AllBands+=UserOneChannelAllBands(UserName,i,Window)
        
    export_data = zip_longest(*AllBands, fillvalue = '')
    with open('training/'+str(Window)+UserName+'train.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
        wr = csv.writer(myfile)
        wr.writerows(export_data)
    myfile.close()

    
def OneUserAllChannelsAllBands(UserName,Window):
    AllBands=[]
    for i in range(32):
        print(i)
        AllBands+=UserOneChannelAllBands(UserName,i,Window)
        
    export_data = zip_longest(*AllBands, fillvalue = '')
    with open('training/'+str(Window)+UserName+"3035"+'train.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
        wr = csv.writer(myfile)
        #wr.writerow((str(ChannelNumber)+"Thetha",str(ChannelNumber)+"Alpha",str(ChannelNumber)+"Beta",str(ChannelNumber)+"Gamma"))
        wr.writerows(export_data)
    myfile.close()


def ParseChannel(ChannelName):
    if ChannelName=="Fp1":
        return 0;
    elif ChannelName=="AF3":
        return 1;
    elif ChannelName=="F3":
         return 2;
    elif ChannelName=="F7":
        return 3;
    elif ChannelName=="FC5":
        return 4;
    elif ChannelName=="FC1":
        return 5;
    elif ChannelName=="C3":
        return 6;
    elif ChannelName=="T7":
        return 7;
    elif ChannelName=="CP5":
        return 8;
    elif ChannelName=="CP1":
        return 9;
    elif ChannelName=="P3":
         return 10;
    elif ChannelName=="P7":
        return 11;
    elif ChannelName=="PO3":
        return 12;
    elif ChannelName=="O1":
        return 13;
    elif ChannelName=="Oz":
        return 14;
    elif ChannelName=="Pz":
        return 15;
    elif ChannelName=="Fp2":
        return 16;
    elif ChannelName=="AF4":
        return 17;
    elif ChannelName=="Fz":
        return 18;
    elif ChannelName=="F4":
        return 19;
    elif ChannelName=="F8":
         return 20;
    elif ChannelName=="FC6":
        return 21;
    elif ChannelName=="FC2":
         return 22;
    elif ChannelName=="Cz":
        return 23;
    elif ChannelName=="C4":
        return 24;
    elif ChannelName=="T8":
        return 25;
    elif ChannelName=="CP6":
        return 26;
    elif ChannelName=="CP2":
        return 27;
    elif ChannelName=="P4":
        return 28;
    elif ChannelName=="P8":
        return 29;
    elif ChannelName=="PO4":
        return 30;
    elif ChannelName=="O2":
        return 31;



   


In [ ]:
if __name__ == "__main__":
    OneUserAllChannelsAllBandsAllVid("s20",256)
    OneUserAllChannelsAllBandsAllVid("s21",256)
    OneUserAllChannelsAllBandsAllVid("s22",256)
    OneUserAllChannelsAllBandsAllVid("s23",256)
    OneUserAllChannelsAllBandsAllVid("s24",256)
    OneUserAllChannelsAllBandsAllVid("s25",256)
    OneUserAllChannelsAllBandsAllVid("s26",256)
    OneUserAllChannelsAllBandsAllVid("s27",256)
    OneUserAllChannelsAllBandsAllVid("s28",256)
    OneUserAllChannelsAllBandsAllVid("s29",256)
    OneUserAllChannelsAllBandsAllVid("s30",256)
    OneUserAllChannelsAllBandsAllVid("s31",256)
    OneUserAllChannelsAllBandsAllVid("s32",256)


    #CheckCorrelation("s01",Array,"Arousal")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
